In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [11]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager
from tmqrfeed.chains import OptionChainList, OptionChain

In [5]:
dm = DataManager()
feed = dm.datafeed
engine = feed.data_engine

In [8]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2012, 1, 1))

In [9]:
chain_values    

,date_end,date_start
ticker,,
US.F.CL.G12.120120,2012-01-18,2011-12-19
US.F.CL.H12.120222,2012-02-20,2012-01-18
US.F.CL.J12.120321,2012-03-19,2012-02-20
US.F.CL.K12.120420,2012-04-18,2012-03-19
US.F.CL.M12.120522,2012-05-18,2012-04-18
US.F.CL.N12.120620,2012-06-18,2012-05-18
US.F.CL.Q12.120720,2012-07-18,2012-06-18
US.F.CL.U12.120822,2012-08-20,2012-07-18
US.F.CL.V12.120920,2012-09-18,2012-08-20


In [13]:
opt_list = OptionChainList(engine.db_get_option_chains('US.F.CL.G12.120120'))

In [15]:
opt_list.find(0)